# Import's

In [1]:
import numpy as np
import os
import pandas as pd
import zipfile
import shutil


# Folder path

In [2]:
zip_folder_path = '/obliczenia' #copy folder path 


# All necessary functions

In [3]:
extract_to_folder = 'unzipped'
def extract_zip_files(zip_folder_path, extract_to_folder):
    """Extracts all zip files from a given folder to a specified folder."""
    extracted_folders = []
    for file in os.listdir(zip_folder_path):
        if file.endswith('.zip'):
            zip_path = os.path.join(zip_folder_path, file)
            extract_path = os.path.join(extract_to_folder, os.path.splitext(file)[0])
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_path)
                extracted_folders.append(extract_path)
    return extracted_folders

extracted_folders = extract_zip_files(zip_folder_path, extract_to_folder)

def read_csv_from_folders(folders, file_name):
    """Reads a specified CSV file from each given folder."""
    data = {}
    for folder in folders:
        full_path = os.path.join(folder, file_name)
        if os.path.isfile(full_path):
            try:
                data[folder] = pd.read_csv(full_path)
            except Exception as e:
                data[folder] = f"Error reading file: {e}"
    return data

def create_comparison_table(data, properties):
    """Creates a comparison table from the provided data."""
    comparison_data = []
    for folder, df in data.items():
        if isinstance(df, pd.DataFrame):
            row = [os.path.basename(folder)]
            for prop in properties:
                filtered_df = df[df['Property'] == prop]
                if not filtered_df.empty:
                    row.append(filtered_df['Value'].iloc[0])
                else:
                    row.append(None)  # Append None if the property is not found
            comparison_data.append(row)
    columns = ['Folder'] + properties
    return pd.DataFrame(comparison_data, columns=columns)


# Properties to compare for each file
basic_properties = ['Molar mass', 'Formula', "Lipinski's rule of five"]
structural_properties = ['Atom count', 'Heavy atom count', 'Asymmetric atom count', 'Rotatable bond count', 'Ring count','Aromatic ring count','FSP3', 'Hydrogen bond donor count', 'Hydrogen bond acceptor count', 'Formal charge', 'Topological polar surface area', 'Polarizability', 'Molar refractivity' ]
lipophilicity_properties = ['HLB', 'logP']

# Creating comparison tables
basic_properties_data = read_csv_from_folders(extracted_folders, 'basic_properties.csv')
basic_comparison_table = create_comparison_table(basic_properties_data, basic_properties)

structural_properties_data = read_csv_from_folders(extracted_folders, 'structural_properties.csv')
structural_comparison_table = create_comparison_table(structural_properties_data, structural_properties)

lipophilicity_data = read_csv_from_folders(extracted_folders, 'lipophilicity.csv')
lipophilicity_comparison_table = create_comparison_table(lipophilicity_data, lipophilicity_properties)

def create_comparison_table_for_solubility(data):
    """Creates a comparison table for solubility properties from the provided data."""
    comparison_data = []
    for folder, df in data.items():
        if isinstance(df, pd.DataFrame):
            average_solubility = df['mg/ml'].mean()
            comparison_data.append([
                os.path.basename(folder), 
                average_solubility
            ])
    return pd.DataFrame(comparison_data, columns=[
        'Folder', 'Average Solubility (mg/ml)'
    ])

def create_comparison_table_for_names_and_identifiers(data):
    """Creates a comparison table for names and identifiers from the provided data."""
    comparison_data = []
    for folder, df in data.items():
        if isinstance(df, pd.DataFrame):
            iupac_name = df[df['Property'] == 'IUPAC name']['Value'].iloc[0] if not df[df['Property'] == 'IUPAC name'].empty else None
            traditional_name = df[df['Property'] == 'Traditional name']['Value'].iloc[0] if not df[df['Property'] == 'Traditional name'].empty else None
            common_names = df[df['Property'] == 'Common names']['Value'].iloc[0] if not df[df['Property'] == 'Common names'].empty else None
            smiles = df[df['Property'] == 'SMILES']['Value'].iloc[0] if not df[df['Property'] == 'SMILES'].empty else None
            inchi = df[df['Property'] == 'InChI']['Value'].iloc[0] if not df[df['Property'] == 'InChI'].empty else None
            comparison_data.append([
                os.path.basename(folder), 
                iupac_name, 
                traditional_name, 
                common_names, 
                smiles, 
                inchi
            ])

    return pd.DataFrame(comparison_data, columns=[
        'Folder', 'IUPAC Name', 'Traditional Name', 'Common Names', 'SMILES', 'InChI'
    ])


def create_comparison_table_for_geometry(data):
    """Creates a comparison table for geometry properties from the provided data."""
    comparison_data = []
    for folder, df in data.items():
        if isinstance(df, pd.DataFrame):
            vdw_volume = df[df['Property'] == 'Van der Waals volume']['Value'].iloc[0] if not df[df['Property'] == 'Van der Waals volume'].empty else None
            vdw_surface_area = df[df['Property'] == 'Van der Waals surface area']['Value'].iloc[0] if not df[df['Property'] == 'Van der Waals surface area'].empty else None
            solvent_accessible_surface_area = df[df['Property'] == 'Solvent accessible surface area']['Value'].iloc[0] if not df[df['Property'] == 'Solvent accessible surface area'].empty else None
            topological_polar_surface_area = df[df['Property'] == 'Topological polar surface area']['Value'].iloc[0] if not df[df['Property'] == 'Topological polar surface area'].empty else None
            min_projection_area = df[df['Property'] == 'Minimum projection area']['Value'].iloc[0] if not df[df['Property'] == 'Minimum projection area'].empty else None
            max_projection_area = df[df['Property'] == 'Maximum projection area']['Value'].iloc[0] if not df[df['Property'] == 'Maximum projection area'].empty else None
            min_projection_radius = df[df['Property'] == 'Minimum projection radius']['Value'].iloc[0] if not df[df['Property'] == 'Minimum projection radius'].empty else None
            max_projection_radius = df[df['Property'] == 'Maximum projection radius']['Value'].iloc[0] if not df[df['Property'] == 'Maximum projection radius'].empty else None
            comparison_data.append([
                os.path.basename(folder), 
                vdw_volume, 
                vdw_surface_area, 
                solvent_accessible_surface_area, 
                topological_polar_surface_area, 
                min_projection_area,
                max_projection_area,
                min_projection_radius,
                max_projection_radius
            ])

    return pd.DataFrame(comparison_data, columns=[
        'Folder', 'Van der Waals Volume (A^3)', 'Van der Waals Surface Area (A^2)', 
        'Solvent Accessible Surface Area (A^2)', 'Topological Polar Surface Area (A^2)', 
        'Minimum Projection Area (A^2)', 'Maximum Projection Area (A^2)', 'Minimum projection radius (A)','Maximum projection radius (A)'
    ])


def create_comparison_table_for_pka(data):
    """Creates a comparison table for pKa properties from the provided data."""
    comparison_data = []
    max_columns = 0

    # Determining the maximum number of microspecies columns across all files
    for df in data.values():
        if isinstance(df, pd.DataFrame):
            max_columns = max(max_columns, df.shape[1] - 1)  # Excluding the 'pH' column

    # Creating the comparison table
    for folder, df in data.items():
        row = [os.path.basename(folder)]
        if isinstance(df, pd.DataFrame):
            for i in range(1, max_columns + 1):  # Starting from 1 to exclude the 'pH' column
                if i < df.shape[1]:
                    # Average of the microspecies column
                    avg_value = df.iloc[:, i].str.rstrip('%').astype(float).mean()
                    row.append(avg_value)
                else:
                    row.append(None)
        else:
            # If the pKa file is not present, fill the row with None
            row.extend([None] * max_columns)

        comparison_data.append(row)

    # Generating column names
    columns = ['Folder'] + [f'Microspecies #{i}' for i in range(1, max_columns + 1)]
    return pd.DataFrame(comparison_data, columns=columns)


def create_comparison_table_for_logd(data):
    """Creates a comparison table for logD properties from the provided data."""
    comparison_data = []
    for folder, df in data.items():
        row = [os.path.basename(folder)]
        if isinstance(df, pd.DataFrame):
            average_logd = df['logD'].mean()
            row.append(average_logd)
        else:
            row.append(None)  # If the logD file is not present, fill the row with None

        comparison_data.append(row)

    return pd.DataFrame(comparison_data, columns=['Folder', 'Average logD'])

def create_comparison_table_for_hnmr(data):
    """Creates a comparison table for HNMR properties from the provided data."""
    comparison_data = []
    for folder, df in data.items():
        row = [os.path.basename(folder)]
        if isinstance(df, pd.DataFrame):
            average_shift = df['Shift'].str.rstrip(' ppm').astype(float).mean()
            row.append(average_shift)
        else:
            row.append(None)  # If the hnmr file is not present, fill the row with None

        comparison_data.append(row)

    return pd.DataFrame(comparison_data, columns=['Folder', 'Average HNMR Shift (ppm)'])

def create_detailed_hnmr_table(data):
    """Creates a detailed table for HNMR data from the provided data."""
    detailed_data = []
    for folder, df in data.items():
        folder_name = os.path.basename(folder)
        if isinstance(df, pd.DataFrame):
            # Add all HNMR data from the folder
            for index, row in df.iterrows():
                detailed_row = [
                    folder_name,
                    row['Atoms'],
                    float(row['Shift'].rstrip(' ppm')),
                    row['Intensity'],
                    row['Multiplet'],
                    row['Quality']
                ]
                detailed_data.append(detailed_row)
        else:
            # If the hnmr file is not present, add a row with None values
            detailed_data.append([folder_name, None, None, None, None, None])

    columns = ['Folder', 'Atoms', 'Shift (ppm)', 'Intensity', 'Multiplet', 'Quality']
    return pd.DataFrame(detailed_data, columns=columns)


def create_comparison_table_for_lipophilicity_additional(data):
    """Creates a comparison table for additional lipophilicity properties from the provided data."""
    comparison_data = []
    for folder, df in data.items():
        row = [os.path.basename(folder)]
        if isinstance(df, pd.DataFrame):
            hlb = df[df['Property'] == 'HLB']['Value'].iloc[0] if not df[df['Property'] == 'HLB'].empty else None
            logp = df[df['Property'] == 'logP']['Value'].iloc[0] if not df[df['Property'] == 'logP'].empty else None
            row.extend([hlb, logp])
        else:
            row.extend([None, None])  # If the lipophilicity file is not present, fill the row with None

        comparison_data.append(row)

    return pd.DataFrame(comparison_data, columns=['Folder', 'HLB', 'logP'])

def create_comparison_table_for_additional_solubility(data):
    """Creates a comparison table for additional solubility properties."""
    comparison_data = []
    for folder, df in data.items():
        row = [os.path.basename(folder)]
        if isinstance(df, pd.DataFrame):
            average_solubility = df['mg/ml'].mean()
            row.append(average_solubility)
        else:
            row.append(None)  # If the solubility file is not present, fill the row with None

        comparison_data.append(row)

    return pd.DataFrame(comparison_data, columns=['Folder', 'Average Solubility (mg/ml)'])

def create_comparison_table_for_isoelectric_point(data):
    """Creates a comparison table for isoelectric point properties."""
    comparison_data = []
    for folder, df in data.items():
        row = [os.path.basename(folder)]
        if isinstance(df, pd.DataFrame):
            # Calculating the isoelectric point where charge is closest to zero
            isoelectric_point = df.iloc[(df['Charge']-0).abs().argsort()[:1]]['pH'].values[0]
            row.append(isoelectric_point)
        else:
            row.append(None)  # If the file is not present, fill the row with None

        comparison_data.append(row)

    return pd.DataFrame(comparison_data, columns=['Folder', 'Isoelectric Point (pH)'])

def create_detailed_hnmr_table(data):
    """Creates a detailed table for HNMR data from the provided data."""
    detailed_data = []
    for folder, df in data.items():
        folder_name = os.path.basename(folder)
        if isinstance(df, pd.DataFrame):
            # Add all HNMR data from the folder
            for index, row in df.iterrows():
                detailed_row = [
                    folder_name,
                    row['Atoms'],
                    float(row['Shift'].rstrip(' ppm')),
                    row['Intensity'],
                    row['Multiplet'],
                    row['Quality']
                ]
                detailed_data.append(detailed_row)
        else:
            # If the hnmr file is not present, add a row with None values
            detailed_data.append([folder_name, None, None, None, None, None])

    columns = ['Folder', 'Atoms', 'Shift (ppm)', 'Intensity', 'Multiplet', 'Quality']
    return pd.DataFrame(detailed_data, columns=columns)


def extract_all_data_from_isoelectric_point(data):
    """Extracts all data from isoelectric_point.csv files."""
    all_data = []
    for folder, df in data.items():
        folder_name = os.path.basename(folder)
        if isinstance(df, pd.DataFrame):
            for index, row in df.iterrows():
                all_data.append([
                    folder_name,
                    row['pH'],
                    row['Charge']
                ])
        else:
            all_data.append([folder_name, None, None])

    return pd.DataFrame(all_data, columns=['Folder', 'pH', 'Charge'])




def extract_all_data_from_solubility(data):
    """Extracts all data from isoelectric_point.csv files."""
    all_data = []
    for folder, df in data.items():
        folder_name = os.path.basename(folder)
        if isinstance(df, pd.DataFrame):
            for index, row in df.iterrows():
                all_data.append([
                    folder_name,
                    row['pH'],
                    row['mg/ml']
                ])
        else:
            all_data.append([folder_name, None, None])

    return pd.DataFrame(all_data, columns=['Folder', 'pH', 'mg/ml'])




def extract_all_data_from_logd(data):
    """Extracts all data from isoelectric_point.csv files."""
    all_data = []
    for folder, df in data.items():
        folder_name = os.path.basename(folder)
        if isinstance(df, pd.DataFrame):
            for index, row in df.iterrows():
                all_data.append([
                    folder_name,
                    row['pH'],
                    row['logD']
                ])
        else:
            all_data.append([folder_name, None, None])

    return pd.DataFrame(all_data, columns=['Folder', 'pH', 'logD'])




def extract_all_data_from_pka(data):
    """Extracts all data from isoelectric_point.csv files."""
    all_data = []
    for folder, df in data.items():
        folder_name = os.path.basename(folder)
        if isinstance(df, pd.DataFrame):
            for index, row in df.iterrows():
                all_data.append([
                    folder_name,
                    row['pH'],
                    row['Microspecies #1'],
                    row['Microspecies #2'],
                 """    row['Microspecies #3'],
                    row['Microspecies #4'],
                    row['Microspecies #5'],
                    row['Microspecies #6'],
                    row['Microspecies #7'] """
                ])
        else:
            all_data.append([folder_name, None, None])

    return pd.DataFrame(all_data, columns=['Folder', 'pH', 'Microspecies #1', 'Microspecies #2', 'Microspecies #3'])

def load_data(file_path):
    return pd.read_csv(file_path)

def reshape_data(df, value_column):
    df_long = df.melt(id_vars='Folder', var_name='pH_Type', value_name=value_column)
    df_long['pH'] = df_long['pH_Type'].apply(lambda x: float(x.split('_')[1]))
    df_long['Type'] = df_long['pH_Type'].apply(lambda x: x.split('_')[0])
    df_long = df_long.drop('pH_Type', axis=1)
    return df_long

def transform_and_save_data(file_path, output_file_path, group_column, value_columns):
    """
    Loads data from a CSV file, groups it, creates new columns for each value,
    drops the original columns, and saves the transformed data to a new CSV file.
    """
    data = pd.read_csv(file_path)
    grouped_data = data.groupby(group_column).agg(list).reset_index()

    max_length = grouped_data[value_columns].applymap(len).max().max()
    for i in range(max_length):
        for col in value_columns:
            grouped_data[f'{col}_{i}'] = grouped_data[col].apply(lambda x: x[i] if i < len(x) else None)
    
    transformed_data = grouped_data.drop(value_columns, axis=1)
    transformed_data.to_csv(output_file_path, index=False)

def make_unique(df):
    """
    Ensures that each combination of Folder and pH is unique.
    It pivots the table so that each Folder has a single row with columns for each pH value.
    """
    # Melting the DataFrame to long format
    df_long = df.melt(id_vars='Folder', var_name='pH', value_name='Value')

    # Splitting the 'pH' column to separate the pH value and the measurement type (Charge, logD, mg/ml)
    df_long[['pH', 'Type']] = df_long['pH'].str.rsplit('_', n=1, expand=True)

    # Pivoting to wide format with one row per folder and one column per pH value and type
    df_wide = df_long.pivot_table(index='Folder', columns=['pH', 'Type'], values='Value').reset_index()

    return df_wide

In [4]:
# Merging the tables
solubility_data = read_csv_from_folders(extracted_folders, 'solubility.csv')
solubility_comparison_table = create_comparison_table_for_solubility(solubility_data)

names_and_identifiers_data = read_csv_from_folders(extracted_folders, 'names_and_identifiers.csv')
names_and_identifiers_comparison_table = create_comparison_table_for_names_and_identifiers(names_and_identifiers_data)

geometry_data = read_csv_from_folders(extracted_folders, 'geometry.csv')
geometry_comparison_table = create_comparison_table_for_geometry(geometry_data)

logd_data = read_csv_from_folders(extracted_folders, 'logD.csv')
logd_comparison_table = create_comparison_table_for_logd(logd_data)

hnmr_data = read_csv_from_folders(extracted_folders, 'hnmr.csv')
hnmr_comparison_table = create_comparison_table_for_hnmr(hnmr_data)
detailed_hnmr_data = create_detailed_hnmr_table(hnmr_data)

lipophilicity_additional_data = read_csv_from_folders(extracted_folders, 'lipophilicity.csv')
lipophilicity_additional_comparison_table = create_comparison_table_for_lipophilicity_additional(lipophilicity_additional_data)

additional_solubility_data = read_csv_from_folders(extracted_folders, 'solubility.csv')
additional_solubility_comparison_table = create_comparison_table_for_additional_solubility(additional_solubility_data)

isoelectric_point_data = read_csv_from_folders(extracted_folders, 'isoelectric_point.csv')
isoelectric_point_comparison_table = create_comparison_table_for_isoelectric_point(isoelectric_point_data)



# Initial merge with basic comparison tables
final_combined_table = basic_comparison_table
final_combined_table = pd.merge(final_combined_table, structural_comparison_table, on='Folder', how='outer')
final_combined_table = pd.merge(final_combined_table, lipophilicity_comparison_table, on='Folder', how='outer')
final_combined_table = pd.merge(final_combined_table, solubility_comparison_table, on='Folder', how='outer')

# Merging additional tables
final_combined_table = pd.merge(final_combined_table, names_and_identifiers_comparison_table, on='Folder', how='outer')
final_combined_table = pd.merge(final_combined_table, geometry_comparison_table, on='Folder', how='outer')
final_combined_table = pd.merge(final_combined_table, logd_comparison_table, on='Folder', how='outer')
final_combined_table = pd.merge(final_combined_table, lipophilicity_additional_comparison_table, on='Folder', how='outer')
final_combined_table = pd.merge(final_combined_table, additional_solubility_comparison_table, on='Folder', how='outer')



In [5]:
folder_name = "data"
folder_path = os.path.join(os.getcwd(), folder_name)
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)

folder_name_2 = "detailed"
folder_path_2 = os.path.join(os.getcwd(), folder_name_2)
if os.path.exists(folder_path_2):
    shutil.rmtree(folder_path_2)
os.makedirs(folder_path)
os.makedirs(folder_path_2)

# Adding analysis for hnmr.csv
hnmr_data = read_csv_from_folders(extracted_folders, 'hnmr.csv')
hnmr_comparison_table = create_comparison_table_for_hnmr(hnmr_data)
detailed_hnmr_data = create_detailed_hnmr_table(hnmr_data)

# Saving detailed HNMR data to a separate CSV file
detailed_hnmr_output_path = 'hnmr_data.csv'
detailed_hnmr_data.to_csv(detailed_hnmr_output_path, index=False)

# For pKa data
all_pka_data_path = 'detailed/transformed_pka_data.csv'

# For logD data
all_logd_data = extract_all_data_from_logd(logd_data)
all_logd_data_path = 'data/all_logd_data.csv'
all_logd_data.to_csv(all_logd_data_path, index=False)

# For solubility data
all_solubility_data = extract_all_data_from_solubility(additional_solubility_data)
all_solubility_data_path = 'data/all_solubility_data.csv'
all_solubility_data.to_csv(all_solubility_data_path, index=False)

# For isoelectric point data
all_isoelectric_point_data = extract_all_data_from_isoelectric_point(isoelectric_point_data)
all_isoelectric_point_data_path = 'data/all_isoelectric_point_data.csv'
all_isoelectric_point_data.to_csv(all_isoelectric_point_data_path, index=False)


In [6]:
# Applying the function to each data set
#transform_and_save_data('pKa.csv', 'transformed_pKa_data.csv', 'Folder', ['pH', 'pKa'])
transform_and_save_data('data/all_logD_data.csv', 'detailed/transformed_logD_data.csv', 'Folder', ['pH', 'logD'])
transform_and_save_data('data/all_solubility_data.csv', 'detailed/transformed_solubility_data.csv', 'Folder', ['pH', 'mg/ml'])


/var/folders/b5/vt5tbdrd3vsb9sjq0cdwyc900000gn/T/ipykernel_4728/1275758066.py:393: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped_data[f'{col}_{i}'] = grouped_data[col].apply(lambda x: x[i] if i < len(x) else None)
/var/folders/b5/vt5tbdrd3vsb9sjq0cdwyc900000gn/T/ipykernel_4728/1275758066.py:393: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped_data[f'{col}_{i}'] = grouped_data[col].apply(lambda x: x[i] if i < len(x) else None)
/var/folders/b5/vt5tbdrd3vsb9sjq0cdwyc900000gn/T/ipykernel_4728/1275758066.py:393: Perform

In [7]:
# Load the uploaded CSV file
file_path = 'data/all_isoelectric_point_data.csv'
data = pd.read_csv(file_path)

# Grouping the data by 'Folder'
grouped_data = data.groupby('Folder').agg(list).reset_index()

# Finding the maximum number of pH values in any folder
max_length = grouped_data[['pH', 'Charge']].applymap(len).max().max()

# Creating new columns for each pH and Charge value
for i in range(max_length):
    grouped_data[f'pH_{i}'] = grouped_data['pH'].apply(lambda x: x[i] if i < len(x) else None)
    grouped_data[f'Charge_{i}'] = grouped_data['Charge'].apply(lambda x: x[i] if i < len(x) else None)

# Dropping the original 'pH' and 'Charge' columns
transformed_table_data = grouped_data.drop(['pH', 'Charge'], axis=1)

# Saving the transformed data to a new CSV file
output_file_path = 'detailed/transformed_isoelectric_point_data.csv'
transformed_table_data.to_csv(output_file_path, index=False)
# Function for loading data


# Paths to files
file_path_isoelectric = 'detailed/transformed_isoelectric_point_data.csv'
file_path_logD = 'detailed/transformed_logD_data.csv'
file_path_solubility = 'detailed/transformed_solubility_data.csv'

# Loading data
isoelectric_point_data = load_data(file_path_isoelectric)
logD_data = load_data(file_path_logD)
solubility_data = load_data(file_path_solubility)

# Transforming data
isoelectric_point_long = reshape_data(isoelectric_point_data, 'Charge')
logD_long = reshape_data(logD_data, 'logD')
solubility_long = reshape_data(solubility_data, 'Solubility')


/var/folders/b5/vt5tbdrd3vsb9sjq0cdwyc900000gn/T/ipykernel_4728/1793275115.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped_data[f'Charge_{i}'] = grouped_data['Charge'].apply(lambda x: x[i] if i < len(x) else None)
/var/folders/b5/vt5tbdrd3vsb9sjq0cdwyc900000gn/T/ipykernel_4728/1793275115.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped_data[f'pH_{i}'] = grouped_data['pH'].apply(lambda x: x[i] if i < len(x) else None)
/var/folders/b5/vt5tbdrd3vsb9sjq0cdwyc900000gn/T/ipykernel_4728/1793275115.py:14: Perfor

In [8]:
final_combined_table.head()

,Folder,Molar mass,Formula,Lipinski's rule of five,Atom count,Heavy atom count,Asymmetric atom count,Rotatable bond count,Ring count,Aromatic ring count,...,Solvent Accessible Surface Area (A^2),Topological Polar Surface Area (A^2),Minimum Projection Area (A^2),Maximum Projection Area (A^2),Minimum projection radius (A),Maximum projection radius (A),Average logD,HLB_y,logP_y,Average Solubility (mg/ml)_y
0,calculation-result-20240105-112340,331.347,C17H18FN3O3,true,42.0,24.0,0.0,3.0,4.0,2.0,...,476.26,72.88,43.14,96.57,5.02,7.48,-1.53113,13.416,-0.768,179.632922


In [9]:
output_file_path = 'combined_properties.csv'
final_combined_table.to_csv(output_file_path, index=False)

# Path for loading the CSV file
file_path = 'combined_properties.csv'

# Loading the data
df = pd.read_csv(file_path)

keywords = ['Name', 'SMILES', 'InChI']
# Identifying columns containing the word 'Name'
name_columns = [col for col in df.columns if any(keyword in col for keyword in keywords)]

# Adding columns with names at the beginning of the DataFrame
reordered_df = df[name_columns + [col for col in df.columns if col not in name_columns]]

# Saving the modified dataframe to a new CSV file
output_file_path = 'combined_properties.csv'
reordered_df.to_csv(output_file_path, index=False)


In [10]:
""" # Load the CSV file
file_path = 'all_isoelectric_point_data.csv'
data = pd.read_csv(file_path)

# Aggregating the data for each folder (compound)
aggregated_data = data.groupby('Folder').agg(['mean', 'median', 'min', 'max'])['Charge']

# Resetting index for better presentation
aggregated_data = aggregated_data.reset_index()

# Displaying the aggregated table
print(aggregated_data.head())
output_file_path = 'agregated.csv'
aggregated_data.to_csv(output_file_path, index=False)
 """

" # Load the CSV file\nfile_path = 'all_isoelectric_point_data.csv'\ndata = pd.read_csv(file_path)\n\n# Aggregating the data for each folder (compound)\naggregated_data = data.groupby('Folder').agg(['mean', 'median', 'min', 'max'])['Charge']\n\n# Resetting index for better presentation\naggregated_data = aggregated_data.reset_index()\n\n# Displaying the aggregated table\nprint(aggregated_data.head())\noutput_file_path = 'agregated.csv'\naggregated_data.to_csv(output_file_path, index=False)\n "

In [11]:
# Load the uploaded CSV file
file_path = 'data/all_isoelectric_point_data.csv'
data = pd.read_csv(file_path)

# Grouping the data by 'Folder'
grouped_data = data.groupby('Folder').agg(list).reset_index()

# Finding the maximum number of pH values in any folder
max_length = grouped_data[['pH', 'Charge']].applymap(len).max().max()

# Creating new columns for each pH and Charge value
for i in range(max_length):
    grouped_data[f'pH_{i}'] = grouped_data['pH'].apply(lambda x: x[i] if i < len(x) else None)
    grouped_data[f'Charge_{i}'] = grouped_data['Charge'].apply(lambda x: x[i] if i < len(x) else None)

# Dropping the original 'pH' and 'Charge' columns
transformed_table_data = grouped_data.drop(['pH', 'Charge'], axis=1)

# Saving the transformed data to a new CSV file
output_file_path = 'detailed/transformed_isoelectric_point_data.csv'
transformed_table_data.to_csv(output_file_path, index=False)

/var/folders/b5/vt5tbdrd3vsb9sjq0cdwyc900000gn/T/ipykernel_4728/156232598.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped_data[f'Charge_{i}'] = grouped_data['Charge'].apply(lambda x: x[i] if i < len(x) else None)
/var/folders/b5/vt5tbdrd3vsb9sjq0cdwyc900000gn/T/ipykernel_4728/156232598.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped_data[f'pH_{i}'] = grouped_data['pH'].apply(lambda x: x[i] if i < len(x) else None)
/var/folders/b5/vt5tbdrd3vsb9sjq0cdwyc900000gn/T/ipykernel_4728/156232598.py:14: Performan

In [12]:

# Applying the function to each data set
#transform_and_save_data('pKa.csv', 'transformed_pKa_data.csv', 'Folder', ['pH', 'pKa'])
transform_and_save_data('data/all_logD_data.csv', 'detailed/transformed_logD_data.csv', 'Folder', ['pH', 'logD'])
transform_and_save_data('data/all_solubility_data.csv', 'detailed/transformed_solubility_data.csv', 'Folder', ['pH', 'mg/ml'])


/var/folders/b5/vt5tbdrd3vsb9sjq0cdwyc900000gn/T/ipykernel_4728/1275758066.py:393: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped_data[f'{col}_{i}'] = grouped_data[col].apply(lambda x: x[i] if i < len(x) else None)
/var/folders/b5/vt5tbdrd3vsb9sjq0cdwyc900000gn/T/ipykernel_4728/1275758066.py:393: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped_data[f'{col}_{i}'] = grouped_data[col].apply(lambda x: x[i] if i < len(x) else None)
/var/folders/b5/vt5tbdrd3vsb9sjq0cdwyc900000gn/T/ipykernel_4728/1275758066.py:393: Perform

In [13]:
# Applying transformations to each table
isoelectric_point_unique = make_unique(isoelectric_point_data)  # Isoelectric point data table
logD_unique = make_unique(logD_data)  # logD data table
solubility_unique = make_unique(solubility_data)  # Solubility data table

# Merging the transformed tables
merged_unique_df = isoelectric_point_unique.merge(logD_unique, on='Folder', how='outer')
merged_unique_df = merged_unique_df.merge(solubility_unique, on='Folder', how='outer')

# Correcting pH column names to range from 0.0 to 14.0
corrected_columns = ['Folder']
for i in range(141):
    pH_value = i / 10  # Correcting pH values
    corrected_columns.extend([f'pH_{pH_value}', f'Charge_{pH_value}', f'solubility_{pH_value}(mg/ml)', f'logD_{pH_value}'])

corrected_df = pd.DataFrame(columns=corrected_columns)

# Filling in the new DataFrame
for index, row in merged_unique_df.iterrows():
    new_row = {'Folder': row['Folder']}
    for i in range(141):
        pH_value = i / 10  # Correcting pH values
        new_row[f'pH_{pH_value}'] = pH_value
        new_row[f'Charge_{pH_value}'] = row['Charge'][i]
        new_row[f'solubility_{pH_value}(mg/ml)'] = row['mg/ml'][i]
        new_row[f'logD_{pH_value}'] = row['logD'][i]
    corrected_df = corrected_df.append(new_row, ignore_index=True)

# Saving the corrected DataFrame to a new CSV file
corrected_csv_file_path = 'data/corrected_reorganized_merged_data.csv'
corrected_df.to_csv(corrected_csv_file_path, index=False)


/var/folders/b5/vt5tbdrd3vsb9sjq0cdwyc900000gn/T/ipykernel_4728/3141446893.py:7: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  merged_unique_df = isoelectric_point_unique.merge(logD_unique, on='Folder', how='outer')
/var/folders/b5/vt5tbdrd3vsb9sjq0cdwyc900000gn/T/ipykernel_4728/3141446893.py:8: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  merged_unique_df = merged_unique_df.merge(solubility_unique, on='Folder', how='outer')
/var/folders/b5/vt5tbdrd3vsb9sjq0cdwyc900000gn/T/ipykernel_4728/3141446893.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  corrected_df = corrected_df.append(new_row, ignore_index=True)


In [14]:
# Loading the datasets
corrected_df = pd.read_csv('data/corrected_reorganized_merged_data.csv')
combined_properties_df = pd.read_csv('combined_properties.csv')

# Cleaning the 'Folder' column in the corrected dataframe
corrected_df['Folder'] = corrected_df['Folder'].str.extract(r'(calculation-result-\d{8}-\d{6})')

# Merging the datasets
columns_to_merge = ['Folder', 'Traditional Name', 'IUPAC Name']
merged_df = pd.merge(corrected_df, combined_properties_df[columns_to_merge], on='Folder', how='left')

# Adjusting the position of 'Traditional Name' and 'IUPAC Name' columns
cols_to_move = ['Traditional Name', 'IUPAC Name']
new_columns_order = ['Folder'] + cols_to_move + [col for col in merged_df.columns if col not in cols_to_move and col != 'Folder']
adjusted_df = merged_df[new_columns_order]

# Saving the adjusted dataframe
adjusted_file_path = 'adjusted_merged_corrected_combined_properties.csv'
adjusted_df.to_csv(adjusted_file_path, index=False)

In [15]:
merged_unique_df

pH                                Folder  Charge                          \
Type                                           0       1      10     100   
0     calculation-result-20240105-112340  1.3829  1.3301  1.0584 -0.9533   

pH                                         ...   pH                           \
Type     101   102    103     104     105  ...   90   91   92   93   94   95   
0    -0.9626 -0.97 -0.976 -0.9809 -0.9848  ...  9.0  9.1  9.2  9.3  9.4  9.5   

pH                        
Type   96   97   98   99  
0     9.6  9.7  9.8  9.9  

[1 rows x 847 columns]